In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time
import math

#Keeping mean fixed sampling from uniform distribution
def generate_random_numbers_with_fixed_average(n, a, b, target_average):
    # Generate n-1 random numbers in the specified range
    random_numbers = np.random.uniform(a, b, n-1)
    
    # Calculate the last number required to achieve the target average
    last_number = n * target_average - np.sum(random_numbers)
    
    # Ensure the last number is within the specified range
    if a <= last_number <= b:
        random_numbers = np.append(random_numbers, last_number)
        return random_numbers
    else:
        # If the last number is outside the range, regenerate the numbers
        return generate_random_numbers_with_fixed_average(n, a, b, target_average)


initial_inv = 0

monthly_con = 50000
n_inv = 20
percent_hike = 0.04

rate = 12
rate_std = 4
rate_list = np.random.normal(rate,rate_std,n_inv)
rate_list_pm = (pow(1+rate_list/100, 1/12)-1)*100 #Changing percentage rate into per month

inf_rate = 6.5
inf_rate_std = 2.5
inf_rate_list = np.random.normal(inf_rate,inf_rate_std,n_inv)

rate_cons = 8

year_amount_withdrawn = 600000

for x in range(n_inv):
    year_amount_withdrawn = year_amount_withdrawn*(1+inf_rate_list[x]/100)
print('Your monthly expense at the end of', n_inv, 'years will be', year_amount_withdrawn/12)

In [ ]:
print("Average inflation rate for this simulation is",inf_rate_list.mean())
print("Average growth rate for this simulation is",rate_list.mean())

In [ ]:
total = initial_inv
for i in range(n_inv):
    monthly_con = monthly_con*(1+percent_hike)
    
    for j in range(12):
        total = (total+monthly_con)*(1+rate_list_pm[i]/100)    

In [ ]:
total_investment = monthly_con*12*n_inv+initial_inv
print('Total amount invested -', total_investment)
print('Total amount in investments -',total)
print('Latest monthly contribution -',monthly_con)

In [ ]:
import dash
from dash.dependencies import Output, Input
from dash import dcc
from dash import html
import plotly
import random
import plotly.graph_objs as go
# from collections import deque

X = []
Y = []
global total
years = 0
# X = deque(maxlen=20)
X.append(years)
# Y = deque(maxlen=20)
Y.append(total)

app = dash.Dash(__name__)
app.layout = html.Div(
    [
        dcc.Graph(id='live-graph', animate=True),
        dcc.Interval(
            id='graph-update',
            interval=1*1000
        ),
    ]
)

@app.callback(Output('live-graph', 'figure'),
              [Input('graph-update', 'n_intervals')])
def update_graph_scatter(input_data):
    global total, years, year_amount_withdrawn
    while total > 0: 
        X.append(X[-1]+1)
        
        total = total - year_amount_withdrawn
        total = total*(1+rate_cons/100)
        Y.append(total)
        year_amount_withdrawn = year_amount_withdrawn*(1+np.random.normal(inf_rate,inf_rate_std)/100) 

        data = plotly.graph_objs.Scatter(
                x=list(X),
                y=list(Y),
                name='Scatter',
                mode= 'lines+markers'
                )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(rangeselector=dict(buttons=list([dict(step="all")])),
                                                    rangeslider=dict(visible=False)),
                                                    yaxis=dict(range=[min(Y),max(Y)]))}


if __name__ == '__main__':
    app.run_server(debug=False)

In [ ]:
year_amount_withdrawn/12

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time
import math
import dash
from dash.dependencies import Output, Input, State
from dash import dcc
from dash import html
import plotly
import random
import plotly.graph_objs as go
# Global variables for storing the state
X = []
Y = []
total = 0
years = 0
simulation_running = False

monthly_contribution = 0
yearly_withdrawal = 0

app = dash.Dash(__name__)

app.layout = html.Div(
    [
        html.H1("Investment Growth Simulation"),
        html.Div([
            html.Label("Initial Investment:"),
            dcc.Input(id='initial_inv', type='number', value=0, step=1000),
        ]),
        html.Div([
            html.Label("Monthly Contribution:"),
            dcc.Input(id='monthly_con', type='number', value=50000, step=1000),
        ]),
        html.Div([
            html.Label("Invest monthly for (years):"),
            dcc.Input(id='n_inv', type='number', value=20, step=1),
        ]),
        html.Div([
            html.Label("Annual Contribution Hike (%):"),
            dcc.Input(id='percent_hike', type='number', value=0.04, step=0.01),
        ]),
        html.Div([
            html.Label("Annual Rate of Return (%):"),
            dcc.Input(id='rate', type='number', value=12, step=0.1),
        ]),
        html.Div([
            html.Label("Rate of return-Standard Deviation (%):"),
            dcc.Input(id='rate_std', type='number', value=4, step=0.1),
        ]),
        html.Div([
            html.Label("Inflation Rate (%):"),
            dcc.Input(id='inf_rate', type='number', value=6.5, step=0.1),
        ]),
        html.Div([
            html.Label("Inflation Rate Standard Deviation (%):"),
            dcc.Input(id='inf_rate_std', type='number', value=2.5, step=0.1),
        ]),
        html.Div([
            html.Label("Conservative Growth Rate (after investment period) (%):"),
            dcc.Input(id='rate_cons', type='number', value=8, step=0.1),
        ]),
        html.Div([
            html.Label("Initial Annual Withdrawal Amount:"),
            dcc.Input(id='year_amount_withdrawn', type='number', value=600000, step=10000),
        ]),
        html.Button('Start Simulation', id='start-button', n_clicks=0),
        html.Button('Reset Simulation', id='reset-button', n_clicks=0),
        dcc.Graph(id='live-graph', animate=True),
        dcc.Interval(
            id='graph-update',
            interval=1*500,
            n_intervals=0
        ),
    ]
)

@app.callback(
    [Output('live-graph', 'figure'),
     Output('start-button', 'n_clicks'),
     Output('reset-button', 'n_clicks')],
    [Input('graph-update', 'n_intervals'),
     Input('start-button', 'n_clicks'),
     Input('reset-button', 'n_clicks')],
    [State('initial_inv', 'value'),
     State('monthly_con', 'value'),
     State('n_inv', 'value'),
     State('percent_hike', 'value'),
     State('rate', 'value'),
     State('rate_std', 'value'),
     State('inf_rate', 'value'),
     State('inf_rate_std', 'value'),
     State('rate_cons', 'value'),
     State('year_amount_withdrawn', 'value')]
)
def update_graph_scatter(n_intervals, start_clicks, reset_clicks,
                         initial_inv, monthly_con, n_inv, percent_hike,
                         rate, rate_std, inf_rate, inf_rate_std,
                         rate_cons, year_amount_withdrawn):
    global X, Y, total, years, simulation_running, monthly_contribution, yearly_withdrawal
    
    if monthly_contribution == 0:
        monthly_contribution = monthly_con
    if yearly_withdrawal == 0:
        yearly_withdrawal = year_amount_withdrawn
    total_investment = initial_inv
    
    

    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = None
        
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'reset-button':
        X.clear()
        Y.clear()
        total = 0
        years = 0
        yearly_withdrawal = 0
        monthly_contribution = 0
        simulation_running = False
        return {'data': [], 'layout': go.Layout()}, 0, 0

    if button_id == 'start-button' and not simulation_running:

        total = initial_inv
        years = 0
        
        rate_list = np.random.normal(rate,rate_std,n_inv)
        rate_list_pm = (pow(1+rate_list/100, 1/12)-1)*100 #Changing percentage rate into per month
        inf_rate_list = np.random.normal(inf_rate, inf_rate_std, n_inv)
        
            
        for i in range(n_inv): 
            monthly_contribution = monthly_contribution*(1+percent_hike)
            total_investment = total_investment + 12*monthly_contribution

            for j in range(12): #Corpus growing
                total = (total+monthly_contribution)*(1+rate_list_pm[i]/100)   

        
        for x in range(n_inv): #Finding the yearly withdrawal amaount at the end of investment period due to inflation
            yearly_withdrawal = yearly_withdrawal*(1+inf_rate_list[x]/100)
        
        X.append(years)
        Y.append(total)

        simulation_running = True

    if simulation_running:
        while total > 0 :
            
            X.append(X[-1] + 1)
            total = total - yearly_withdrawal
            total = total*(1 + rate_cons/100)
            Y.append(total)
            yearly_withdrawal = yearly_withdrawal*(1+np.random.normal(inf_rate,inf_rate_std)/100) #yearly withdrawel keeps increasing with time
            print(yearly_withdrawal)
            print(total)


            data = go.Scatter(
                x=list(X),
                y=list(Y),
                name='Scatter',
                mode='lines+markers'
            )

            return {'data': [data], 'layout': go.Layout(
                xaxis=dict(rangeselector=dict(buttons=list([dict(step="all")])),
                           rangeslider=dict(visible=False)),
                yaxis=dict(range=[0.83*min(Y), 1.2*max(Y)])
            )}, 0, 0
    
    else:
        return {'data': [], 'layout': go.Layout()}, 0, 0

if __name__ == '__main__':
    app.run_server(debug=False)
